In [ ]:
def geodesic_line_waypoints(loc_1, loc_2, n_waypoints):
    all_points = []
    l = geod.InverseLine(loc_1.latitude, loc_1.longitude, loc_2.latitude, loc_2.longitude)
    print(l.s13 / 1000)
    ds = l.s13 / (n_waypoints - 1)
    print(ds)
    for i in range(n_waypoints - 1):
        s = min(ds * i, l.s13)
        g = l.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        all_points.append(Location(g['lat2'], g['lon2']))
    all_points.append(Location(loc_2.latitude, loc_2.longitude))
    return all_points

initial_waypoints = geodesic_line_waypoints(loc_1, loc_2, 9)

In [ ]:
from typing import List
import numpy as np

def distance_to_degree(distance):
    return distance / geod.a

def bearing2coords(bearing, lat, lon, distance):
    bearing = np.deg2rad(bearing)
    la1 = np.deg2rad(lat)
    lo1 = np.deg2rad(lon)
    Ad = distance_to_degree(distance)
    la2 =  np.arcsin(np.sin(la1) * np.cos(Ad)  + np.cos(la1) * np.sin(Ad) * np.cos(bearing))
    lo2 = lo1 + np.arctan2(np.sin(bearing) * np.sin(Ad )* np.cos(la1) , np.cos(Ad) - np.sin(la1) * np.sin(la2))
    x1 = np.rad2deg(la2)
    y1=np.rad2deg(lo2)
    return x1, y1

def generate_new_route(waypoints: List[Location], deviation):
    start_wp = waypoints[0]
    end_wp = waypoints[-1]
    bearing = geod.Inverse(start_wp.latitude, start_wp.longitude, end_wp.latitude, end_wp.longitude)['azi1']
    new_bearing = bearing + deviation
    new_route = []
    new_start = Location(*bearing2coords(new_bearing, start_wp.latitude, start_wp.longitude))
    new_end = Location(*bearing2coords(-new_bearing, end_wp.latitude, end_wp.longitude))
    new_l = geodesic_line_waypoints(new_start, new_end, 7)
#     for waypoint in waypoints[:-2]:
#         lat, lon = waypoint
#         new_lat, new_lon = bearing2coords(new_bearing, lat, lon)
#         new_route.append([new_lat, new_lon])abs
#     new_route = [new_start, new_end]
    return new_l

def generate_new_route_2(waypoints: List[Location], deviation, distance):
    start_wp = waypoints[0]
    end_wp = waypoints[-1]
    new_route = []
    for start_wp in waypoints:
        
        bearing = geod.Inverse(start_wp.latitude, start_wp.longitude, end_wp.latitude, end_wp.longitude)['azi1']
        new_bearing = bearing + deviation
        new_start = Location(*bearing2coords(new_bearing, start_wp.latitude, start_wp.longitude, distance))
        new_route.append(new_start)
#     new_end = Location(*bearing2coords(-new_bearing, end_wp.latitude, end_wp.longitude))
#     new_l = geodesic_line_waypoints(new_start, new_end, 7)
#     for waypoint in waypoints[:-2]:
#         lat, lon = waypoint
#         new_lat, new_lon = bearing2coords(new_bearing, lat, lon)
#         new_route.append([new_lat, new_lon])abs
#     new_route = [new_start, new_end]
    return new_route

